# Assignment 8: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A8-2-data.zip](A7-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [1]:
import pandas as pd

import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__

# list all rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
orders = pd.read_csv("orders.csv")
order_products = pd.read_csv("order_products.csv")
users = pd.read_csv("users.csv")

# could drop columns here, if needed

print(users["label"].value_counts())
print(orders.shape, order_products.shape)
users.head()

False    628
True     139
Name: label, dtype: int64
(5997, 4) (57780, 8)


,Unnamed: 0,user_id,label
0,0,1,False
1,1,2,True
2,2,3,False
3,3,7,False
4,4,10,False


In [3]:
orders.head()

,Unnamed: 0,order_id,user_id,order_time
0,0,2539329,1,2015-01-01 08:00:00
1,1,2398795,1,2015-01-16 15:00:00
2,2,473747,1,2015-02-07 03:00:00
3,3,2254736,1,2015-03-08 10:00:00
4,11,2168274,2,2015-01-01 11:00:00


In [4]:
order_products.head()

,Unnamed: 0,order_id,reordered,product_name,aisle_id,department,order_time,order_product_id
0,86,230811,0,Organic 1% Low Fat Milk,84,dairy eggs,2015-01-01 18:00:00,230811_3
1,87,230811,0,Lowfat Vanilla Yogurt,120,dairy eggs,2015-01-01 18:00:00,230811_6
2,88,230811,0,Organic American Cheese Singles,21,dairy eggs,2015-01-01 18:00:00,230811_22
3,89,230811,0,Organic Shredded Parmesan,21,dairy eggs,2015-01-01 18:00:00,230811_24
4,90,230811,0,Chocolate Lowfat Milk,84,dairy eggs,2015-01-01 18:00:00,230811_21


## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [16]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    #load entities into entity-set
    es = ft.EntitySet(id = 'Order')
    es = es.entity_from_dataframe(entity_id= 'users', dataframe= users,
                                  index = 'user_id')
    es = es.entity_from_dataframe(entity_id = 'orders', dataframe = orders, 
                                  index = 'order_id', time_index = 'order_time')
    es = es.entity_from_dataframe(entity_id= 'order_products', dataframe= order_products,
                                  index = 'order_product_id', time_index= 'order_time')
    
    #generate relationship
    rel_user_order = ft.Relationship(es['users']['user_id'],
                                        es['orders']['user_id'])
    es = es.add_relationship(rel_user_order)
    rel_order_product = ft.Relationship(es['orders']['order_id'],
                                        es['order_products']['order_id'])
    es = es.add_relationship(rel_order_product)
    
    return es

### 1.2 Deep Feature Synthesis

In [39]:
# Automatically generate features
es = load_entityset(orders, order_products, users)
# use ft.dfs to perform feature engineering
feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'users')

In [41]:
# output what features you generate
# shape (767,162），over a hundred of new features generated
feature_matrix

,Unnamed: 0,label,COUNT(orders),MAX(orders.Unnamed: 0),MEAN(orders.Unnamed: 0),MIN(orders.Unnamed: 0),SKEW(orders.Unnamed: 0),STD(orders.Unnamed: 0),SUM(orders.Unnamed: 0),COUNT(order_products),MAX(order_products.Unnamed: 0),MAX(order_products.aisle_id),MAX(order_products.reordered),MEAN(order_products.Unnamed: 0),MEAN(order_products.aisle_id),MEAN(order_products.reordered),MIN(order_products.Unnamed: 0),MIN(order_products.aisle_id),MIN(order_products.reordered),MODE(order_products.department),MODE(order_products.product_name),NUM_UNIQUE(order_products.department),NUM_UNIQUE(order_products.product_name),SKEW(order_products.Unnamed: 0),SKEW(order_products.aisle_id),SKEW(order_products.reordered),STD(order_products.Unnamed: 0),STD(order_products.aisle_id),STD(order_products.reordered),SUM(order_products.Unnamed: 0),SUM(order_products.aisle_id),SUM(order_products.reordered),MAX(orders.COUNT(order_products)),MAX(orders.MEAN(order_products.Unnamed: 0)),MAX(orders.MEAN(order_products.aisle_id)),MAX(orders.MEAN(order_products.reordered)),MAX(orders.MIN(order_products.Unnamed: 0)),MAX(orders.MIN(order_products.aisle_id)),MAX(orders.MIN(order_products.reordered)),MAX(orders.NUM_UNIQUE(order_products.department)),MAX(orders.NUM_UNIQUE(order_products.product_name)),MAX(orders.SKEW(order_products.Unnamed: 0)),MAX(orders.SKEW(order_products.aisle_id)),MAX(orders.SKEW(order_products.reordered)),MAX(orders.STD(order_products.Unnamed: 0)),MAX(orders.STD(order_products.aisle_id)),MAX(orders.STD(order_products.reordered)),MAX(orders.SUM(order_products.Unnamed: 0)),MAX(orders.SUM(order_products.aisle_id)),MAX(orders.SUM(order_products.reordered)),MEAN(orders.COUNT(order_products)),MEAN(orders.MAX(order_products.Unnamed: 0)),MEAN(orders.MAX(order_products.aisle_id)),MEAN(orders.MAX(order_products.reordered)),MEAN(orders.MEAN(order_products.Unnamed: 0)),MEAN(orders.MEAN(order_products.aisle_id)),MEAN(orders.MEAN(order_products.reordered)),MEAN(orders.MIN(order_products.Unnamed: 0)),MEAN(orders.MIN(order_products.aisle_id)),MEAN(orders.MIN(order_products.reordered)),MEAN(orders.NUM_UNIQUE(order_products.department)),MEAN(orders.NUM_UNIQUE(order_products.product_name)),MEAN(orders.SKEW(order_products.Unnamed: 0)),MEAN(orders.SKEW(order_products.aisle_id)),MEAN(orders.SKEW(order_products.reordered)),MEAN(orders.STD(order_products.Unnamed: 0)),MEAN(orders.STD(order_products.aisle_id)),MEAN(orders.STD(order_products.reordered)),MEAN(orders.SUM(order_products.Unnamed: 0)),MEAN(orders.SUM(order_products.aisle_id)),MEAN(orders.SUM(order_products.reordered)),MIN(orders.COUNT(order_products)),MIN(orders.MAX(order_products.Unnamed: 0)),MIN(orders.MAX(order_products.aisle_id)),MIN(orders.MAX(order_products.reordered)),MIN(orders.MEAN(order_products.Unnamed: 0)),MIN(orders.MEAN(order_products.aisle_id)),MIN(orders.MEAN(order_products.reordered)),MIN(orders.NUM_UNIQUE(order_products.department)),MIN(orders.NUM_UNIQUE(order_products.product_name)),MIN(orders.SKEW(order_products.Unnamed: 0)),MIN(orders.SKEW(order_products.aisle_id)),MIN(orders.SKEW(order_products.reordered)),MIN(orders.STD(order_products.Unnamed: 0)),MIN(orders.STD(order_products.aisle_id)),MIN(orders.STD(order_products.reordered)),MIN(orders.SUM(order_products.Unnamed: 0)),MIN(orders.SUM(order_products.aisle_id)),MIN(orders.SUM(order_products.reordered)),MODE(orders.DAY(order_time)),MODE(orders.MODE(order_products.department)),MODE(orders.MODE(order_products.product_name)),MODE(orders.MONTH(order_time)),MODE(orders.WEEKDAY(order_time)),MODE(orders.YEAR(order_time)),NUM_UNIQUE(orders.DAY(order_time)),NUM_UNIQUE(orders.MODE(order_products.department)),NUM_UNIQUE(orders.MODE(order_products.product_name)),NUM_UNIQUE(orders.MONTH(order_time)),NUM_UNIQUE(orders.WEEKDAY(order_time)),NUM_UNIQUE(orders.YEAR(order_time)),SKEW(orders.COUNT(order_products)),SKEW(orders.MAX(order_products.Unnamed: 0)),SKEW(orders.MAX(order_products.aisle_id)),SKEW(orders.MAX(order_products.reordered)),SKEW(orders.MEAN(order_

## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [82]:
# --- Write your code below ---
# Select top-10 features and return X, y (X.shape = (767, 10))
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# generate data
# need to remove string type feature for selection
X = feature_matrix.drop(columns=['label']).drop(columns='MODE(orders.MODE(order_products.product_name))')\
    			  .drop(columns='MODE(orders.MODE(order_products.department))')\
                  .drop(columns ='MODE(order_products.product_name)')\
                  .drop(columns ='MODE(order_products.department)')
#back up X before selection
ori_X = X
# remove nan
X = X.fillna(0)
y = feature_matrix['label']

# method1: PCA method yields 0.62 acc 
# But using PCA cannot output the original features because the top components are linear combination of original vars
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

# method2: Univariate selection yields 0.67 acc
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
selector = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_sel = sel.fit_transform(X)

ksel = SelectKBest(f_classif, k=10)
X_uni = ksel.fit_transform(X_sel, y)

# warp up
X =X_uni


### 2.2 Get accuracy and list features

In [85]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

# Print top-10 features
print()
print("top 10 features are: ")
columns = ori_X.columns
cols = ksel.get_support()
for index, c in enumerate(cols):
    if c:
        print(columns[index])


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.67

top 10 features are: 
MEAN(order_products.reordered)
MIN(order_products.Unnamed: 0)
MAX(orders.SKEW(order_products.aisle_id))
MAX(orders.SUM(order_products.reordered))
MEAN(orders.COUNT(order_products))
MEAN(orders.MAX(order_products.reordered))
MEAN(orders.MEAN(order_products.reordered))
MEAN(orders.MIN(order_products.aisle_id))
MEAN(orders.SKEW(order_products.reordered))
MEAN(orders.SUM(order_products.reordered))


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

--- Write your answer here---
### Advantages
1. A wrap-up API for generating features like mean, variant, max, and other aggregate saves us a lot of time for manual coding.
2. The 'entity' and 'entity set' concepts also help us to easily link multiple tables together when generating new features, while manual coding might be pretty troublesome.
3. The number of new features generated (over 100 new features in this case) is pretty large. This is almost impossible for humans to do.

### Disadvantages
1. The large amount of new features generated makes it impossible for humans to check one by one, so the selection part is entirely relying on other algorithms.
2. Some of the features generated by featuretools are useless, which does not help the prediction part.
3. It is okay to manually set up three entities, but when there are lots of tables, manually setting up would be tiring.

### Solutions
1. This is not something that can be solved unless we have the patience to go through the features.
2. I did a general feature generation in this task, but if we want to generate more reasonable features, it is better to assign some features like max, min, and other aggregation processes. But this falls back to generating new features manually. So I would say, the solution is to adjust the feature selection part carefully.
3. Add a wrap-up API for identifying relationships between entities by checking common columns in different tables.

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 8.